In [1]:
import nltk               #used for data pre processing...text data-'.',',',filler words..library stopwords function remove all common stop words in paragragh
from nltk.corpus import stopwords
import numpy as np          #for wighted average of numbers
import networkx as nx                          #to represent in graph
from gensim.models import word2vec                                  #for word to vector
import gensim.downloader as api                 #to download pre trained models
from gensim import models


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
text = """An emotionally fit and stable person always feels vibrant and truly alive and can easily manage emotionally difficult situations. To be emotionally strong, one has to be physically fit too. Although mental health is a personal issue, what affects one person may or may not affect another; yet, several key elements lead to mental health issues.Mental fitness implies a state of psychological well-being. It denotes having a positive sense of how we feel, think, and act, which improves one’s ability to enjoy life. Mental health means keeping our minds healthy. Mankind generally is more focused on keeping their physical body healthy. People tend to ignore the state of their minds. Human superiority over other animals lies in his superior mind. Man has been able to control life due to his highly developed brain. So, it becomes very important for a man to keep both his body and mind fit and healthy. Both physical and mental health are equally important for better performance and results.Mental fitness implies a state of psychological well-being. It denotes having a positive sense of how we feel, think, and act, which improves one’s ability to enjoy life. It contributes to one’s inner ability to be self-determined. It is a proactive, positive term and forsakes negative thoughts that may come to mind. The term mental fitness is increasingly being used by psychologists, mental health practitioners, schools, organisations, and the general population to denote logical thinking, clear comprehension, and reasoning ability"""

In [31]:
text

'An emotionally fit and stable person always feels vibrant and truly alive and can easily manage emotionally difficult situations. To be emotionally strong, one has to be physically fit too. Although mental health is a personal issue, what affects one person may or may not affect another; yet, several key elements lead to mental health issues.Mental fitness implies a state of psychological well-being. It denotes having a positive sense of how we feel, think, and act, which improves one’s ability to enjoy life. Mental health means keeping our minds healthy. Mankind generally is more focused on keeping their physical body healthy. People tend to ignore the state of their minds. Human superiority over other animals lies in his superior mind. Man has been able to control life due to his highly developed brain. So, it becomes very important for a man to keep both his body and mind fit and healthy. Both physical and mental health are equally important for better performance and results.Menta

In [32]:
sentences=nltk.sent_tokenize(text)

In [33]:
len(sentences)

15

In [34]:
def preprocess_sentences(s):
    sentence=s.lower()
    words=nltk.word_tokenize(sentence)           #breakdown sentence in words
    words=[word for word in words if word not in set(stopwords.words('english'))]
    return words

preprocessed_sentences=[preprocess_sentences(sentence) for sentence in sentences]

In [35]:
preprocessed_sentences[1]

['emotionally', 'strong', ',', 'one', 'physically', 'fit', '.']

In [8]:
#wv_model=models.KeyedVectors.load_word2vec_format(r"GoogleNews-vectors-negative300.bin.gz",binary=True,limit=500000)
wv_model=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [36]:
def sentSim(s1,s2):
    v1=np.mean([wv_model[word] for word in s1 if word in wv_model],axis=0)
    v2=np.mean([wv_model[word] for word in s2 if word in wv_model],axis=0)
    if np.all(v1) and np.all(v2):
        return 1-np.dot(v1,v2)/np.linalg.norm(v1)*np.linalg.norm(v2)
    else:
        return 0.0
smm=np.zeros((len(sentences),len(sentences)))
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i!=j:
            smm[i][j]=sentSim(preprocessed_sentences[i],preprocessed_sentences[j])

In [37]:
smm

array([[ 0.        , -0.88627124,  0.40418106, -0.001899  , -0.15747201,
        -0.03295135,  0.01486075,  0.13774508,  0.25715542, -0.20500803,
         0.11043835, -0.001899  ,  0.0338816 ,  0.12239987,  0.31036222],
       [-0.40408206,  0.        ,  0.42717689,  0.25112766, -0.05648804,
         0.01922148,  0.32691091,  0.24497265,  0.38988405, -0.0979774 ,
         0.03673965,  0.25112766,  0.26600254,  0.35085076,  0.3660503 ],
       [ 0.10345101, -0.15795565,  0.        ,  0.05904013, -0.58810294,
        -0.09156656, -0.11726034,  0.02815837,  0.24184293, -0.0150696 ,
        -0.4431802 ,  0.05904013, -0.15011835, -0.00929689, -0.10976112],
       [-0.11037886, -0.11497891,  0.30695951,  0.        , -0.19491577,
         0.02822042, -0.07192576, -0.00813997,  0.36183679, -0.05969167,
         0.0265432 , -0.55867493, -0.28501749, -0.10395765,  0.16796792],
       [ 0.07274276, -0.13701439,  0.1545096 ,  0.13626635,  0.        ,
        -0.34758353, -0.20571601, -0.00608993, 

In [38]:
smg=nx.from_numpy_array(smm)

In [39]:
#scores=nx.pagerank(smg)
#scores=nx.pagerank(smg,max_iter=99, tol=1e-6)
hub_scores, scores= nx.hits(smg)

In [40]:
scores

{0: 0.04969765857854312,
 1: -0.042030247259056235,
 2: 0.15602415511702353,
 3: 0.06359065511521952,
 4: -0.10357976825528482,
 5: 0.06562829646253256,
 6: 0.08915344046936949,
 7: 0.06821774970292067,
 8: 0.1486749673481119,
 9: 0.09642730694286722,
 10: 0.09093742426439885,
 11: 0.04973350968063846,
 12: 0.11664258369845272,
 13: 0.09445140962619353,
 14: 0.056430858508069566}

In [41]:
num=2
#sorting acc to most imp and least
sortSen=sorted(((scores[i],sentence) for i,sentence in enumerate(sentences)),reverse=True)

In [42]:
sortSen

[(0.15602415511702353,
  'Although mental health is a personal issue, what affects one person may or may not affect another; yet, several key elements lead to mental health issues.Mental fitness implies a state of psychological well-being.'),
 (0.1486749673481119,
  'Man has been able to control life due to his highly developed brain.'),
 (0.11664258369845272,
  'It contributes to one’s inner ability to be self-determined.'),
 (0.09642730694286722,
  'So, it becomes very important for a man to keep both his body and mind fit and healthy.'),
 (0.09445140962619353,
  'It is a proactive, positive term and forsakes negative thoughts that may come to mind.'),
 (0.09093742426439885,
  'Both physical and mental health are equally important for better performance and results.Mental fitness implies a state of psychological well-being.'),
 (0.08915344046936949, 'People tend to ignore the state of their minds.'),
 (0.06821774970292067,
  'Human superiority over other animals lies in his superior 

In [43]:
summ_sent=[sentence for _,sentence in sortSen[:num]]

In [44]:
summ_sent


['Although mental health is a personal issue, what affects one person may or may not affect another; yet, several key elements lead to mental health issues.Mental fitness implies a state of psychological well-being.',
 'Man has been able to control life due to his highly developed brain.']

In [45]:
summ="".join(summ_sent)

In [49]:
print(summ)


Although mental health is a personal issue, what affects one person may or may not affect another; yet, several key elements lead to mental health issues.Mental fitness implies a state of psychological well-being.Man has been able to control life due to his highly developed brain.
